# About this Notebook

The goal of this notebook is to build a DL classifier to find toxic comments. The data has been taken from a series of Kaggle competitions to classify Wikipedia comments as toxic/nontoxic. The data has been sourced from Google and Jigsaw. 

Though the full dataset includes non-English comments, I will restrict myself to English-only comment for this iteration. 

I will explore deep learning approaches, using a combination of pretrained word embeddings and simple deep learning models like RNNs and 1D convolutions to do more benchmarking. 

Next, we will explore deep learning models that have 'memory' using LSTMs (Long Short Term Memory) and GRUs (Gated Recurrent Units). 

Finally, we will approach state of the art performance using pretrained models like BERT and xlnet.

For metrics, I will focus on both ROC and precision-recall curves. In addition, I will look at the confusion matrix and performance across different flavors of toxicity.

Credits:
- https://www.kaggle.com/tanulsingh077/deep-learning-for-nlp-zero-to-transformers-bert
- https://www.kaggle.com/jagangupta/stop-the-s-toxic-comments-eda
- https://www.kaggle.com/clinma/eda-toxic-comment-classification-challenge
- https://www.kaggle.com/abhi111/naive-bayes-baseline-and-logistic-regression

My approach to feature engineering and building the model is below:

Deep Learning:
1. Use standard tokenizers and compare with 'homegrown' version from above.
2. Use open source word embeddings for corpus as input to RNN models. Quantify how misspellings affect the standard tokenizers.
3. Find way to input additional features like punctuation/capitalization from approach above to Deep Learning RNN models.
4. Try progressively more complicated deep learning sequence models approaching SOTA.
5. Use metrics from above.

Potential Modules:
1. Correct misspellings
2. Analytics for preprocessing
3. Analytics for model performance (use multi-labels, make easy way to look at specific examples)
4. Automatically generate a lookup table for common variations of words (particularly toxic words, e.g., 'mothafucka' -> 'motherfucker')




In [1]:
import numpy as np
import pandas as pd 
from collections import defaultdict as ddict, Counter
from itertools import compress
from tqdm import tqdm
from scipy.sparse import csr_matrix, hstack
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import re

import random
import string
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   
pd.options.display.max_rows = 999

In [2]:
from toxicity import constants, data, features, text_preprocessing, model, metrics, visualize


[nltk_data] Downloading package stopwords to /Users/jkc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jkc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jkc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jkc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/jkc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jkc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load data

In [3]:
input_data_path = './../'+constants.INPUT_PATH
df_train = data.load(input_data_path, filter=False)

## Define Metrics

In [ ]:
metrics.

In [4]:
def run_metrics(predictions, predictions_prob, target, visualize=True):
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions_prob)
    roc_auc = metrics.auc(fpr, tpr)
    precision, recall, thresholds = metrics.precision_recall_curve(target, predictions_prob)
    average_precision = metrics.average_precision_score(yvalid, pred)
    #average_recall = metrics.recall_score(yvalid, pred)
    print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))
    accuracy = metrics.accuracy_score(yvalid, pred)
    print(metrics.confusion_matrix(yvalid, pred, labels=[0,1]))
    print("Accuracy Score: {0:0.2f}".format(accuracy))
    if visualize:
        plt.figure()
        plt.plot(fpr, tpr)
        plt.title('ROC curve, AUC: {0:0.2f}'.format(roc_auc))
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        
        plt.show()
        
        plt.figure()
        plt.plot(recall, precision)
        plt.title('Precision-Recall curve')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.show()
        #disp = metrics.plot_precision_recall_curve(nb_classifier, count_valid, yvalid)
        #disp.ax_.set_title('2-class Precision-Recall curve: '
                   #'AP={0:0.2f}'.format(average_precision))

In [5]:
xtrain, xvalid, ytrain, yvalid = model.make_train_test(df_train)

In [6]:
len(xvalid)

44710

## Use Deep Learning

In [7]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping, History, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam



## Preprocess data

### We will check the maximum number of words that can be present in a comment , this will help us in padding later

In [8]:
max_len = int(round(df_train['comment_text'].apply(lambda x:len(str(x).split())).max(), -2)+100)
print("Max length of comment text is: {}".format(max_len))

Max length of comment text is: 2400


### First do Tokenization of input corpus

In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
token_toxic = text.Tokenizer(num_words=None)
token_nontoxic = text.Tokenizer(num_words=None)

token.fit_on_texts(list(xtrain) + list(xvalid))
token_toxic.fit_on_texts(train.comment_text.values[train.toxic==1])
token_nontoxic.fit_on_texts(train.comment_text.values[train.toxic==0])

xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
word_toxic = token_toxic.word_index
word_nontoxic = token_nontoxic.word_index

In [ ]:
print(len(word_toxic))
print(len(word_nontoxic))

Example for fitting tokenizer line-by-line if corpus is too big to fit into memory

with open('/Users/liling.tan/test.txt') as fin: for line in fin:
t.fit_on_texts(line.split()) # Fitting the tokenizer line-by-line.

M = []

with open('/Users/liling.tan/test.txt') as fin: for line in fin:

    # Converting the lines into matrix, line-by-line.
    m = t.texts_to_matrix([line], mode='count')[0]
    M.append(m)

## Use pretrained word embeddings

## Convert our one-hot word index into semantic rich GloVe vectors

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open(pre_path + 'glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:


words_not_in_corpus = ddict(int)
words_in_corpus = ddict(int)
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_nontoxic.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        words_in_corpus[word]+=1
    else:
        words_not_in_corpus[word]+=1

In [ ]:
print(len(words_not_in_corpus))
print(len(words_in_corpus))
max(words_not_in_corpus.values())
max(words_in_corpus.values())

#For the full dataset, more than half the 'words' are not found in the glove embeddings
#For the 10K sample dataset, only ~25% of the words are not found in the glove embeddings


In [ ]:
print(len(words_not_in_corpus))
print(len(words_in_corpus))
max(words_not_in_corpus.values())
max(words_in_corpus.values())

#For the full dataset, more than half the 'words' are not found in the glove embeddings
#For the 10K sample dataset, only ~25% of the words are not found in the glove embeddings


In [ ]:
#Save embeddings so they can be easily loaded
np.save('/kaggle/working/glove_embedding_for_full_data', embedding_matrix)

In [ ]:
#Load embeddings
embedding_matrix = np.load('/kaggle/working/glove_embedding_for_10K_sample.npy')

In [ ]:
embedding_matrix.shape

## Simple RNN Model

In [ ]:
opt = Adam(learning_rate=0.0001)

In [ ]:
model1 = Sequential()
model1.add(Embedding(len(word_index) + 1,
                 300,
                 input_length=max_len))
model1.add(SimpleRNN(100))
model1.add(Dense(1, activation='relu'))
model1.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
model1.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
EPOCHS = 10
checkpoint_filepath = '/kaggle/working/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)


my_callbacks = [
    model_checkpoint_callback,
    TensorBoard(log_dir='/kaggle/working/logs'),
    EarlyStopping(monitor='val_loss', patience=3)
]
model_checkpoint_callback

In [ ]:
model1.fit(xtrain_pad, 
           ytrain, 
           epochs=50, 
           batch_size=100, 
           callbacks=my_callbacks,
           validation_split=0.2,)

In [ ]:
scores = model1.predict(xvalid_pad)[:, 0]
preds = scores>.5
run_metrics(preds, scores, yvalid)

## Simple LSTM Model

In [ ]:
%%time
# A simple LSTM with glove embeddings and one dense layer
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

In [ ]:
model.fit(xtrain_pad, 
          ytrain, 
          epochs=50, 
          batch_size=100,
          callbacks=my_callbacks,
          validation_split=0.2,)

In [ ]:
scores = model.predict(xvalid_pad)
preds = scores>.5
run_metrics(preds, scores, yvalid)

# Summary

So far, with very little preprocessing, we have achieved high accuracy. This is a little bit misleading however because the training set is highly imbalanced (roughly 10% positive/toxic class). 

Slightly older techniques, bag-of-words and tf-idf have done better than a simple deep learning models out-of-the-box. This can been seen by the higher AUCs and accuracy of these models in contrast to the simple RNN model. In addition, training these models was extremely fast, even on a local machine. In contrast, the deep learning models required more than 10 minutes to train even five epochs. In addition, trainingg the simple RNN required playing around with the learning rate to get network to learn. The first few attempts produced labels of all zeros. 

The simple LSTM model starts to improve dramatically over the simple RNN model even with only 5 epochs, showing that using the semantic rich word embeddings and including memory already improve simple deep learning results. Though the overall accuracy has decreased in the LSTM model vs the Naive Bayes models, the AUC and precision-recall and ROC curves are much better than the simple models. As we approach more state-of-the-art (SOTA) models and move beyond simple proof-of-concept model training, i.e., try different network parameters, experiment with data preprocessing, do hyperparameter optimization, train until the results start to degrade, add regularization, etc., the results will likely improve even more dramatically.


## Try a GRU Model

In [ ]:
%%time
# GRU with glove embeddings and two dense layers
 model = Sequential()
 model.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))
 model.add(SpatialDropout1D(0.3))
 model.add(GRU(300))
 model.add(Dense(1, activation='sigmoid'))

 model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

In [ ]:
%%time
# GRU with glove embeddings and two dense layers
 model = Sequential()
 model.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))
 model.add(SpatialDropout1D(0.3))
 model.add(GRU(300))
 model.add(Dense(1, activation='sigmoid'))

 model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model.summary()

In [ ]:
model.fit(xtrain_pad, ytrain, nb_epoch=5, batch_size=64)

In [ ]:
scores = model.predict(xvalid_pad)


## Bidirectional RNN Model

In [ ]:
%%time
# A simple bidirectional LSTM with glove embeddings and one dense layer
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                 300,
                 weights=[embedding_matrix],
                 input_length=max_len,
                 trainable=False))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model.summary()

In [ ]:
model.fit(xtrain_pad, ytrain, nb_epoch=5, batch_size=64)

In [ ]:
scores = model.predict(xvalid_pad)


## Seq2seq Architecture

In [ ]:
#TBD
